# Define a model in Hidet
On top of running PyTorch and ONNX models through their respective frontends in Hidet, you may also define your own model with Hidet and run it. In this tutorial, we will walk through a simple example of defining, optimizing, and running a small model with Hidet.

In [1]:
import hidet

# set the cache dir 
hidet.option.cache_dir('./hidet-cache')
# clear the operator cache 
hidet.utils.hidet_clear_op_cache()

Clearing operator cache: /home/yaoyao/repos/asplos23-tutorial/hidet_tutorials_ipynb/hidet-cache/ops


### Define the model
Defining a Hidet model is similar to defining a PyTorch model. Instead of the PyTorch operators, we will use Hidet operators.

In [2]:
def conv_relu(x, out_channels, kernel, stride, padding):
    w = hidet.randn(shape=[out_channels, x.shape[1], kernel, kernel]).cuda()
    b = hidet.randn(shape=[1, out_channels, 1, 1]).cuda()
    x = hidet.ops.conv_pad(x, pads=padding)
    x = hidet.ops.conv2d(x, w, stride=stride) + b
    x = hidet.ops.relu(x)
    return x

We define a simple model consisting of a convolution layer and a ReLU layer. Inside the model, we first randomly generate the weight `w` and bias `b` as constants. We use `.cuda()` to move them to the GPU as we later wish to run the model using GPU. Then we get the result by padding the input `x`, and applying the `conv2d` and `relu` layers. For a list of other operators, please refer to the [Hidet Python API](https://docs.hidet.org/stable/python_api/ops/index.html).

### Imperatively run the model

The model can be directly run by feeding in an input tensor and the parameters.

In [3]:
img = hidet.randn([1, 3, 224, 224]).cuda()
out = conv_relu(img, 64, 7, 2, 3)

Compiling cuda task pad(data=float32[1, 3, 224, 224])...
Compiling cuda task conv2d(x=float32[1, 3, 230, 230], w=float32[64, 3, 7, 7])...
Compiling cuda task add(x=float32[1, 64, 112, 112], y=float32[1, 64, 1, 1])...
Compiling cuda task relu(x=float32[1, 64, 112, 112])...


We randomly generate an input `img`. The output tensor of the model is `out`.

### Trace the model and run
A more efficient way to run the model is to first trace the execution and get the static computation graph of the deep learning model.

In [4]:
img_sym = hidet.symbol([1, 3, 224, 224]).cuda()
out_sym = conv_relu(img_sym, 64, 7, 2, 3)
graph = hidet.trace_from(out_sym, inputs=[img_sym])
print(graph)

Graph(x: float32[1, 3, 224, 224]){
  c = Constant(float32[64, 3, 7, 7])
  c_1 = Constant(float32[1, 64, 1, 1])
  x_1: float32[1, 3, 230, 230] = Pad(x, pads=[0, 0, 3, 3, 0, 0, 3, 3], mode="constant", value=0.0)  
  x_2: float32[1, 64, 112, 112] = Conv2d(x_1, c, stride=[2, 2], groups=1, dilations=(1, 1))  
  x_3: float32[1, 64, 112, 112] = Add(x_2, c_1)  
  x_4: float32[1, 64, 112, 112] = Relu(x_3)  
  return x_4
}


We define the input `img_sym` as a symbol tensor. We get the symbol output by running the `conv_relu()` model with the symbol input. `out_sym` is a symbol tensor that contains all the information of how it is derived. Then we can use `hidet.trace_from()` to create the static computation graph from the symbol output. 

`graph` is an instance of `hidet.graph.FlowGraph`, which is Hidet's representation of a computation graph and also the basic unit of graph-level optimizations. You can print `graph` to get its textual definition.

In [5]:
from hidet.utils import benchmark_func
cuda_graph = graph.cuda_graph()
img = hidet.randn([1, 3, 224, 224]).cuda()
(out,) = cuda_graph.run([img])
print('Hidet: {:.3f} ms'.format(benchmark_func(lambda: cuda_graph.run())))

Hidet: 0.304 ms


We can use the `cuda_graph()` method of a `FlowGraph` to create a `CudaGraph`. Cuda Graph is a more effcient way to submit workloads to NVIDIA GPUs as it eliminates framework-side overhead.

We randomly generate an input tensor `img` and get the output tensor `out` by calling `run()` with the cuda graph and input. We also test the latency of our model.
### Optimize the model

In [6]:
hidet.option.search_space(0)
with hidet.graph.PassContext() as ctx:
    ctx.save_graph_instrument('./outs/graphs')
    graph_opt = hidet.graph.optimize(graph)
    cuda_graph_opt = graph_opt.cuda_graph()
    print('Hidet optimized: {:.3f} ms'.format(benchmark_func(lambda: cuda_graph_opt.run())))


Compiling cuda task rearrange(x=float32[1, 64, 3, 7, 7])...
Compiling cuda task batch_matmul(a=float32[1, 12544, 147], b=float32[1, 147, 64], batch_size=1, m_size=12544, n_size=64, k_size=147, mma=simt) (6 fused)...


Hidet optimized: 0.106 ms


To optimize the model, we set the level of operator schedule space to 2 with `hidet.option.search_space()`. The search space level can be : 0, 1, and 2. By default, the search space level is 0, which means no kernel tuning. The higher the level, the better performance but longer compilation time. The above code may take a few minutes to run, depending on the machine.

We also perform graph level optimizations with `hidet.graph.optimize()` and save intermiediate graphs with `save_graph_instrument()`. We run the benchmark again to see that the resulting optimized model runs faster than the previous unoptimized version.

### Inspect the generated code
The source code and compiled kernel by Hidet are cached under hidet's cache director, you could get the path to the cache directory with

In [7]:
hidet.option.get_cache_dir()

'/home/yaoyao/repos/asplos23-tutorial/hidet_tutorials_ipynb/hidet-cache'

The structure of this cache directory are shown in the documentation. For example, it may look like
```text
cache_root
|-- onnx                          (automatically downloaded ONNX models)
|   |-- bert.onnx
|   `-- resnet50.onnx
`-- ops
    |-- cpu_space_0
    |-- cuda_space_0              (<target>_space_<space>)
    |   |-- add
    |   |   `-- 41920731adb3acf4  (task string hash)
    |   |       |-- lib.so        (compiled kernel)
    |   |       |-- nvcc_log.txt  (compilation command and nvcc output)
    |   |       |-- source.cu     (kernel source code)
    |   |       `-- task.txt      (task string)
    |   `-- matmul
    |       `-- 92dfdc1734b3854d
    |           |-- lib.so
    |           |-- nvcc_log.txt
    |           |-- source.cu
    |           `-- task.txt
```

### Example of generated kernel


In [9]:
# print the source code for conv2d without optimization
print(graph.nodes[1].task_func.source(color=True))

#include <stdint.h>
#include <cuda_fp16.h>
#include <cuda_bf16.h>
#include <hidet/runtime/cuda_context.h>
#include <hidet/runtime/cpu_context.h>
typedef float tfloat32_t;
#define __float_to_tf32(x) (x)
/*
Task(
  name: conv2d
  parameters: 
    x: tensor(float32, [1, 3, 230, 230])
    w: tensor(float32, [64, 3, 7, 7])
    out: tensor(float32, [1, 64, 112, 112])
  inputs: [x, w]
  outputs: [out]
  computations: 
    w: tensor(float32, [64, 3, 7, 7])
    x: tensor(float32, [1, 3, 230, 230])
    out: float32[1, 64, 112, 112] where out[v, v_1, v_2, v_3] = reduce([3, 7, 7], (v_4, v_5, v_6) => (x[v, (((v_1 / 64) * 3) + v_4), ((v_2 * 2) + v_5), ((v_3 * 2) + v_6)] * w[v_1, v_4, v_5, v_6]), sumreduce)
  attributes: {}
)
*/
extern "C" {

__global__ void __launch_bounds__(500) hidet_compute_out(float * __restrict__ x, float * __restrict__ w, float * __restrict__ out) {
  if ((((int)blockIdx.x * 500) + (int)threadIdx.x) < 802816) {
    float acc_Sum = 0.0f;
    for (int32_t v = 0; (v < 3); v = (v 

In [10]:
# print out the source code of conv2d with implicit gemm algorithm
print(graph_opt.nodes[-1].task_func.source(color=True))

#include <stdint.h>
#include <cuda_fp16.h>
#include <cuda_bf16.h>
#include <hidet/runtime/cuda_context.h>
#include <hidet/runtime/cpu_context.h>
typedef float tfloat32_t;
#define __float_to_tf32(x) (x)
/*
Task(
  name: batch_matmul
  parameters: 
    b: tensor(float32, [1, 147, 64])
    y: tensor(float32, [1, 64, 1, 1])
    data: tensor(float32, [1, 3, 224, 224])
    y_1: tensor(float32, [1, 64, 112, 112])
  inputs: [a, b_1]
  outputs: [c]
  computations: 
    b_1: tensor(float32, [1, 147, 64])
    a: tensor(float32, [1, 12544, 147])
    c: float32[1, 12544, 64] where c[v, v_1, v_2] = reduce([147], (v_3) => (a[v, v_1, v_3] * b_1[v, v_3, v_2]), sumreduce)
  attributes: {batch_size: 1, m_size: 12544, n_size: 64, k_size: 147, mma: simt}
  task_graph: TaskGraph(b, y, data) {
    out = pad(data_1=data)
      out: float32[1, 3, 230, 230] where out[v_4, v_5, v_6, v_7] = ((((((0 <= v_4) && (v_4 < 1)) && ((0 <= v_5) && (v_5 < 3))) && ((0 <= (v_6 - 3)) && ((v_6 - 3) < 224))) && ((0 <= (v_7 - 3))